# Data Modeling

In [ ]:
import scanpy as sc
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.cluster.hierarchy import fcluster
import pandas as pd
import pycm

In [ ]:
def train_test_split(adata, fraction: float):
    test_idx = adata.obs.sample(frac=fraction, random_state=42).index
    return adata[~adata.obs_names.isin(test_idx)].copy(), adata[test_idx].copy()

In [ ]:
def agglomerative_clustering(
    adata, nclusters: int, groupby: str = "Sample", n_pcs: int = None
):
    if groupby == "Sample" and "Sample" not in adata.obs.columns:
        adata.obs["Sample"] = adata.obs_names.astype("category")
    sc.tl.dendrogram(adata, groupby=groupby)
    labels = data.obs[groupby].cat.categories
    clusters = fcluster(
        adata.uns[f"dendrogram_{groupby}"]["linkage"], t=nclusters, criterion="maxclust"
    )
    clusters = pd.Series(clusters, index=labels).astype("category")
    clusters = clusters.reindex(adata.obs[groupby].values).values
    adata.obs["Cluster"] = clusters

# Daten einlesen

In [ ]:
data = sc.read("processed_data.h5ad")

## Differentielle Expressionsanalyse

In [ ]:
sc.tl.rank_genes_groups(
    data, groupby="ER Status", groups=["ER+"], reference="ER-", method="t-test"
)

In [ ]:
results = sc.get.rank_genes_groups_df(data, group="ER+").query(
    "pvals_adj < 0.05 & abs(logfoldchanges) > 0.585"
)

In [ ]:
results

In [ ]:
sc.pl.clustermap(
    data[:, results.nsmallest(20, "pvals_adj").names],
    obs_keys="ER Status",
    use_raw=False,
)

In [ ]:
sc.tl.rank_genes_groups(
    data, groupby="relapse", groups=["yes"], reference="no", method="t-test"
)

In [ ]:
results = sc.get.rank_genes_groups_df(data, group="yes").query(
    "pvals_adj < 0.05 & abs(logfoldchanges) > 0.585"
)

In [ ]:
results.nsmallest(20, "pvals_adj")

In [ ]:
sc.pl.clustermap(
    data[:, results.nsmallest(20, "pvals_adj").names], obs_keys="relapse", use_raw=False
)

## Clustering

In [ ]:
agglomerative_clustering(data, nclusters=3, groupby="Sample")

In [ ]:
sc.pl.dendrogram(data, groupby="Sample")

In [ ]:
sc.pl.pca(data, color="Cluster")

In [ ]:
sc.pl.clustermap(data, obs_keys="Cluster", use_raw=False)

## Random Forest

In [ ]:
train, test = train_test_split(data[:, results.nsmallest(20, "pvals_adj").names], 0.2)

In [ ]:
rf = RandomForestClassifier(max_features="log2")

In [ ]:
rf.fit(train.X, train.obs.relapse)

In [ ]:
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
tree.plot_tree(rf.estimators_[0],
               feature_names = data.var_names, 
               class_names=data.obs.relapse.cat.categories,
               impurity=False,
               filled = True);

In [ ]:
predicted_train = rf.predict(train.X)

In [ ]:
traincm = pycm.ConfusionMatrix(actual_vector=train.obs.relapse.values.astype(str), predict_vector=predicted_train)
traincm.plot(number_label=True)

In [ ]:
traincm.ACC

In [ ]:
traincm.FPR

In [ ]:
traincm.FNR

In [ ]:
predicted_test = rf.predict(test.X)

In [ ]:
testcm = pycm.ConfusionMatrix(actual_vector=test.obs.relapse.values.astype(str), predict_vector=predicted_test)
testcm.plot(number_label=True)

In [ ]:
testcm.ACC

In [ ]:
testcm.FPR

In [ ]:
testcm.FNR